In [ ]:
## hhayn router1
from typing_extensions import Literal
from langgraph.graph import MessagesState
from typing import TypedDict

# =========================
# 1. State 정의
# =========================
class Router1State(MessagesState):
    # 사용자 입력(전처리된 텍스트)
    question: str
    # HR 질문 여부
    is_hr_question: Literal["yes", "no"]
    # 라우팅 방향
    next_step: Literal["rag", "reject"]

# =========================
# 2. Router1 Node (LLM 기반 판별)
# =========================
def router1_node(state: Router1State) -> Router1State:
    prompt = f"""
    당신은 HR과 관련 있는 질문을 분류하는 어시스턴트입니다.
    사용자의 질문이 HR과 관련 있는지 판단하세요.

    출력 형식은 아래와 같이만 하세요:
    is_hr_question: yes | no
    next_step: rag | reject

    질문: "{state['question']}"
    """

    response = llm.invoke(prompt).content.strip()

    # 기본값 (안정성 확보용)
    is_hr, next_step = "no", "reject"

    for line in response.splitlines():
        if line.startswith("is_hr_question:"):
            is_hr = line.split(":")[1].strip()
        if line.startswith("next_step:"):
            next_step = line.split(":")[1].strip()

    return {
        **state,
        "is_hr_question": is_hr,
        "next_step": next_step,
    }

# =========================
# 3. Router1 Router
# =========================
def route_after_router1(state: Router1State) -> str:
    if state["next_step"] == "rag":
        return "router2"   # HR 관련 질문 → Router2로 이동
    else:
        return "reject"    # HR 무관 → Reject 노드로 이동

# =========================
# 4. Reject Node
# =========================
def reject_node(state: Router1State) -> dict:
    """
    HR 무관 질문에 대해 거절 안내 메시지를 생성하는 노드
    """
    return {
        **state,
        "answer": "❌ 지원하지 않는 질문입니다. HR 관련 문의만 가능합니다."
    }